### Interactivity

In [1]:
import altair as alt
import pandas as pd

cocktails = pd.read_csv('../data/cocktails.tsv', delimiter='\t')

In [2]:
cocktails.head(2)

,name,abv,acid,sugar,type,index,instructions,ingredients,ncotw
0,Pisco Sour,12.1,0.68,7.2,eggwhite,5,"Dry shake, shake with ice, coupe. Add 3 drops ...",2 oz pisco (40% abv)<br/>1 oz egg white<br/>3/...,not yet
1,Pink Lady,12.4,0.64,9.0,eggwhite,6,"Dry shake, shake with ice, coupe.",1 1/2 oz Plymouth gin<br/>1 oz egg white<br/>3...,not yet


In [3]:
alt.Chart(cocktails).mark_circle().encode(x='acid', y='sugar')

alt.Chart(...)

In [4]:
alt.Chart(cocktails).mark_circle().encode(x='acid', y='sugar').interactive()

alt.Chart(...)

Not super interesting, though...

In [5]:
three = cocktails[cocktails['type'].isin(['shaken', 'stirred', 'blended'])]

In [6]:
# common config
brush = alt.selection(type='interval')
base = alt.Chart(three).add_selection(brush)

# points
points = base.mark_circle(size=100).encode(
    x='acid',
    y='sugar',
    color=alt.condition(brush, 'type', alt.value('grey'))
)

points

alt.Chart(...)

In [7]:
base.mark_tick().encode(
    x='acid',
    y='type',
    color=alt.condition(brush, 'type', alt.value('lightgrey'))
)

alt.Chart(...)

In [8]:
base.mark_tick().encode(
    alt.X('sugar', axis=alt.Axis(labels=False, domain=False, ticks=False)),
    alt.Y('type', title='', axis=alt.Axis(labels=False, domain=False, ticks=False)),
    color=alt.condition(brush, 'type', alt.value('lightgrey'))
)

alt.Chart(...)

In [9]:
# common ticks
tick_axis = alt.Axis(labels=False, domain=False, ticks=False)

sugar = base.mark_tick().encode(
    alt.X('sugar', axis=tick_axis),
    alt.Y('type', title='', axis=tick_axis),
    color=alt.condition(brush, 'type', alt.value('lightgrey'))
)

acid = base.mark_tick().encode(
    alt.X('acid', title='', axis=tick_axis),
    alt.Y('type', axis=tick_axis),
    color=alt.condition(brush, 'type', alt.value('lightgrey'))
)

sugar & acid

alt.VConcatChart(...)

In [10]:
# common config
brush = alt.selection(type='interval')
base = alt.Chart(three).add_selection(brush)

# points
points = base.mark_circle(size=100).encode(
    x='acid',
    y='sugar',
    color=alt.condition(brush, 'type', alt.value('grey'))
)

# ticks
tick_axis = alt.Axis(labels=False, domain=False, ticks=False)

sugar = base.mark_tick().encode(
    alt.Y('sugar', axis=tick_axis),
    alt.X('type', title='', axis=tick_axis),
    color=alt.condition(brush, 'type', alt.value('lightgrey'))
)

acid = base.mark_tick().encode(
    alt.Y('type', title='', axis=tick_axis),
    alt.X('acid', axis=tick_axis),
    color=alt.condition(brush, 'type', alt.value('lightgrey'))
)

# all together
sugar | (points & acid)

alt.HConcatChart(...)

#### More Interactivity

In [11]:
marathon = pd.read_csv('../data/marathon.csv')
marathon['time'] = marathon['time'].apply(pd.to_datetime)

In [12]:
alt.Chart(marathon).mark_line().encode(
    x='time',
    y='distance',
    color='runner'
).interactive()

alt.Chart(...)

In [13]:
alt.Chart(marathon).mark_line(size=3).encode(
    x='hour',
    y='distance',
    color='gender',
    detail='runner'
).interactive()

alt.Chart(...)

In [14]:
alt.Chart(marathon).mark_line(size=3).encode(
    x='hour',
    y='distance',
    color='gender',
    detail='runner',
    tooltip='runner'
).interactive()

alt.Chart(...)

In [15]:
selector = alt.selection_single(
    fields=['gender'], 
    empty='all',
    bind='legend'
)

runners = alt.Chart(marathon).mark_line(point=True).encode(
    x='hour',
    y='distance',
    color='gender',
    detail='runner',
    opacity=alt.condition(selector, alt.value(1), alt.value(0))
).add_selection(
    selector
)

runners

alt.Chart(...)

Time to Flex 💪

In [16]:
sex = pd.read_csv('../data/sex.csv')
sex = sex.rename(columns={'age2': 'age'})
sex['pct'] /= 100

chart = (
    alt.Chart(sex)
    .mark_circle()
    .encode(
        x=alt.X('pct:Q', axis=alt.Axis(title='', format='%')),
        y=alt.Y('age', 
            axis=alt.Axis(title='', grid=True),
            scale=alt.Scale(domain=['70+','60-69', '50-59', '40-49', '30-39', '25-29', '18-24']),
        ), 
        color=alt.Color('response', 
            scale=alt.Scale(
                domain=['Not wrong', 'Wrong-ish', 'Wrong'], 
                range=["#39a9db", "#f39237", "#d63230"]),
            legend=alt.Legend(title='', orient='top')
        )
    )
    .properties(height=100, width=100)
    .facet('religion', columns=3)
    .configure_view(strokeWidth=0)
    .properties(
        background='#F0F0F0',
        title='Sex Before Marriage'
    )
)

chart

alt.FacetChart(...)

Strip it down to focus...

In [17]:
alt.Chart(sex).mark_circle().encode(
    x='pct',
    y='age', 
    color=alt.Color('response', 
        scale=alt.Scale(
            domain=['Not wrong', 'Wrong-ish', 'Wrong'], 
            range=["#39a9db", "#f39237", "#d63230"])
    )
).properties(height=100, width=100).facet('religion', columns=3)

alt.FacetChart(...)

In [18]:
selector = alt.selection_single(
    fields=['response'], 
    empty='all',
    bind='legend'
)

chart = (
    alt.Chart(sex)
    .mark_circle()
    .encode(
        x='pct',
        y='age', 
        color=alt.Color('response', 
            scale=alt.Scale(
                domain=['Not wrong', 'Wrong-ish', 'Wrong'], 
                range=["#39a9db", "#f39237", "#d63230"])
        ),
        # add here
        opacity=alt.condition(selector, alt.value(1), alt.value(0))
    )
    .properties(height=100, width=100)
    .facet('religion', columns=3)
    # and here
    .add_selection(selector)
)

chart

alt.FacetChart(...)

Put it all together...

In [19]:
chart = (
    alt.Chart(sex)
    .mark_circle()
    .encode(
        x=alt.X('pct:Q', axis=alt.Axis(title='', format='%')),
        y=alt.Y('age', 
            axis=alt.Axis(title='', grid=True),
            scale=alt.Scale(domain=['70+','60-69', '50-59', '40-49', '30-39', '25-29', '18-24']),
        ), 
        color=alt.Color('response', 
            scale=alt.Scale(
                domain=['Not wrong', 'Wrong-ish', 'Wrong'], 
                range=["#39a9db", "#f39237", "#d63230"]),
            legend=alt.Legend(title='', orient='top')
        ),
        # add here
        opacity=alt.condition(selector, alt.value(1), alt.value(0))
    )
    .properties(height=100, width=100)
    .facet('religion', columns=3)
    .configure_view(strokeWidth=0)
    .properties(
        background='#F0F0F0',
        title='Sex Before Marriage'
    )
    # fix that pesky title
    .configure(
        title=alt.TitleConfig(fontSize=14, anchor='middle', offset=10),
        legend=alt.LegendConfig(labelFontSize=12, titleFontSize=12, symbolSize=100, offset=10))
    # and here
    .add_selection(selector)
)

chart

alt.FacetChart(...)

### Saving the chart

In [20]:
chart.save('../index.html')

In [21]:
with open('../index.html', 'r') as f:
    html = f.read()
    
print(html)

<!DOCTYPE html>
<html>
<head>
  <style>
    .error {
        color: red;
    }
  </style>
  <script type="text/javascript" src="https://cdn.jsdelivr.net/npm//vega@5"></script>
  <script type="text/javascript" src="https://cdn.jsdelivr.net/npm//vega-lite@4.8.1"></script>
  <script type="text/javascript" src="https://cdn.jsdelivr.net/npm//vega-embed@6"></script>
</head>
<body>
  <div id="vis"></div>
  <script>
    (function(vegaEmbed) {
      var spec = {"config": {"view": {"continuousWidth": 400, "continuousHeight": 300}, "legend": {"labelFontSize": 12, "offset": 10, "symbolSize": 100, "titleFontSize": 12}, "title": {"anchor": "middle", "fontSize": 14, "offset": 10}}, "data": {"name": "data-cc39b92d2241f14d033ea7c943f8042d"}, "facet": {"type": "nominal", "field": "religion"}, "spec": {"mark": "circle", "encoding": {"color": {"type": "nominal", "field": "response", "legend": {"orient": "top", "title": ""}, "scale": {"domain": ["Not wrong", "Wrong-ish", "Wrong"], "range": ["#39a9db", "#f3